In [ ]:
!pip install pyod

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
#READ DATA
data = pd.read_csv('anamoly.csv')
data = data.drop(['outlier'], axis=1)

In [34]:
def setup(data, 
          target = None, 
          split = 0.7):
    
    global X, X_train, X_test, y_train, y_test, seed
    
    if target == None:
        X = data
        import random
        seed = random.randint(150,900)
        
    else:
        from sklearn.model_selection import train_test_split
        X = data.drop(target,axis=1)
        y = data[target]
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-split)
        import random
        seed = random.randint(150,900)

In [11]:
def create_model(estimator = None, 
                 fraction = 0.05):

    #ignore warings
    import warnings
    warnings.filterwarnings('ignore') 

    if estimator == 'abod':
        from pyod.models.abod import ABOD
        model = ABOD(contamination=fraction)
    elif estimator == 'cluster':
        from pyod.models.cblof import CBLOF
        model = CBLOF(contamination=fraction, random_state=seed)
    elif estimator == 'cof':
        from pyod.models.cof import COF
        model = COF(contamination=fraction)        
    elif estimator == 'iforest':
        from pyod.models.iforest import IForest
        model = IForest(contamination=fraction, random_state=seed)              
    elif estimator == 'hist':
        from pyod.models.hbos import HBOS
        model = HBOS(contamination=fraction)         
    elif estimator == 'knn':
        from pyod.models.knn import KNN
        model = KNN(contamination=fraction)          
    elif estimator == 'lof':
        from pyod.models.lof import LOF
        model = LOF(contamination=fraction)      
    elif estimator == 'svm':
        from pyod.models.ocsvm import OCSVM
        model = OCSVM(contamination=fraction)
    elif estimator == 'pca':
        from pyod.models.pca import PCA
        model = PCA(contamination=fraction, random_state=seed)    
    elif estimator == 'mcd':
        from pyod.models.mcd import MCD
        model = MCD(contamination=fraction, random_state=seed)
    elif estimator == 'sod':
        from pyod.models.sod import SOD
        model = SOD(contamination=fraction)         
    elif estimator == 'sos':
        from pyod.models.sos import SOS
        model = SOS(contamination=fraction)       
    
    model.fit(X)
    return model

In [77]:
def assign_model(model,
                type = None):
    
    if type == None:
        pred_labels = model.labels_
        pred_score = model.decision_scores_
        X['Label'] = pred_labels
        X['Score'] = pred_score
        return X
    elif type == 'predict':
        pred_labels = model.predict(X)
        pred_score = model.decision_function(X)
        X['Label'] = pred_labels
        X['Score'] = pred_score
        return X        

In [91]:
def plot_model(model):
    pass

In [35]:
setup(data)

In [124]:
abod = create_model('abod')

In [128]:
print(abod)

ABOD(contamination=0.05, method='fast', n_neighbors=5)


In [129]:
df = assign_model(abod)

In [130]:
df.head()

,0,1,2,3,4,5,6,7,8,9,Label,Score
0,0.263995,0.764929,0.138424,0.935242,0.605867,0.518790,0.912225,0.608234,0.723782,0.733591,0,-0.269018
1,0.546092,0.653975,0.065575,0.227772,0.845269,0.837066,0.272379,0.331679,0.429297,0.367422,0,-0.749131
2,0.336714,0.538842,0.192801,0.553563,0.074515,0.332993,0.365792,0.861309,0.899017,0.088600,0,-1.231164
3,0.092108,0.995017,0.014465,0.176371,0.241530,0.514724,0.562208,0.158963,0.073715,0.208463,1,-0.019604
4,0.325261,0.805968,0.957033,0.331665,0.307923,0.355315,0.501899,0.558449,0.885169,0.182754,1,-0.145362
